In [1]:
import pandas as pd
import sqlite3 
from tabulate import tabulate

# Importing Datasets

In [2]:
def print_r(cursor):
    column_names = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    print(tabulate(rows, headers=column_names, tablefmt="pretty"))

In [3]:
df_gs = pd.read_csv("goalscorers.csv")
df_r = pd.read_csv("results.csv")
df_s = pd.read_csv("shootouts.csv")

conn = sqlite3.connect("sports_data.db")

# Maintaining index=False
df_gs.to_sql("goalscorers", conn, if_exists="replace", index=False)
df_r.to_sql("results", conn, if_exists="replace", index=False)
df_s.to_sql("shootouts", conn, if_exists="replace", index=False)

crsr = conn.cursor()

## Goalscorers

In [4]:
crsr.execute("""
    SELECT *
    FROM goalscorers 
    LIMIT 5;
""")

print_r(crsr)

+------------+-----------+-----------+-----------+------------------+--------+----------+---------+
|    date    | home_team | away_team |   team    |      scorer      | minute | own_goal | penalty |
+------------+-----------+-----------+-----------+------------------+--------+----------+---------+
| 1916-07-02 |   Chile   |  Uruguay  |  Uruguay  | José Piendibene  |  44.0  |    0     |    0    |
| 1916-07-02 |   Chile   |  Uruguay  |  Uruguay  | Isabelino Gradín |  55.0  |    0     |    0    |
| 1916-07-02 |   Chile   |  Uruguay  |  Uruguay  | Isabelino Gradín |  70.0  |    0     |    0    |
| 1916-07-02 |   Chile   |  Uruguay  |  Uruguay  | José Piendibene  |  75.0  |    0     |    0    |
| 1916-07-06 | Argentina |   Chile   | Argentina |  Alberto Ohaco   |  2.0   |    0     |    0    |
+------------+-----------+-----------+-----------+------------------+--------+----------+---------+


In [5]:
crsr.execute("PRAGMA table_info(goalscorers);")
table_info = crsr.fetchall()

for column in table_info:
    print(column)

(0, 'date', 'TEXT', 0, None, 0)
(1, 'home_team', 'TEXT', 0, None, 0)
(2, 'away_team', 'TEXT', 0, None, 0)
(3, 'team', 'TEXT', 0, None, 0)
(4, 'scorer', 'TEXT', 0, None, 0)
(5, 'minute', 'REAL', 0, None, 0)
(6, 'own_goal', 'INTEGER', 0, None, 0)
(7, 'penalty', 'INTEGER', 0, None, 0)


## Results

In [6]:
crsr.execute("""
    SELECT *
    FROM results 
    LIMIT 5;
""")

print_r(crsr)

+------------+-----------+-----------+------------+------------+------------+---------+----------+---------+
|    date    | home_team | away_team | home_score | away_score | tournament |  city   | country  | neutral |
+------------+-----------+-----------+------------+------------+------------+---------+----------+---------+
| 1872-11-30 | Scotland  |  England  |     0      |     0      |  Friendly  | Glasgow | Scotland |    0    |
| 1873-03-08 |  England  | Scotland  |     4      |     2      |  Friendly  | London  | England  |    0    |
| 1874-03-07 | Scotland  |  England  |     2      |     1      |  Friendly  | Glasgow | Scotland |    0    |
| 1875-03-06 |  England  | Scotland  |     2      |     2      |  Friendly  | London  | England  |    0    |
| 1876-03-04 | Scotland  |  England  |     3      |     0      |  Friendly  | Glasgow | Scotland |    0    |
+------------+-----------+-----------+------------+------------+------------+---------+----------+---------+


In [7]:
crsr.execute("PRAGMA table_info(results);")
table_info = crsr.fetchall()

for column in table_info:
    print(column)

(0, 'date', 'TEXT', 0, None, 0)
(1, 'home_team', 'TEXT', 0, None, 0)
(2, 'away_team', 'TEXT', 0, None, 0)
(3, 'home_score', 'INTEGER', 0, None, 0)
(4, 'away_score', 'INTEGER', 0, None, 0)
(5, 'tournament', 'TEXT', 0, None, 0)
(6, 'city', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'neutral', 'INTEGER', 0, None, 0)


## Shootouts

In [8]:
crsr.execute("""
    SELECT *
    FROM shootouts
    LIMIT 5;
""")

print_r(crsr)

+------------+-------------+------------------+-------------+---------------+
|    date    |  home_team  |    away_team     |   winner    | first_shooter |
+------------+-------------+------------------+-------------+---------------+
| 1967-08-22 |    India    |      Taiwan      |   Taiwan    |               |
| 1971-11-14 | South Korea | Vietnam Republic | South Korea |               |
| 1972-05-07 | South Korea |       Iraq       |    Iraq     |               |
| 1972-05-17 |  Thailand   |   South Korea    | South Korea |               |
| 1972-05-19 |  Thailand   |     Cambodia     |  Thailand   |               |
+------------+-------------+------------------+-------------+---------------+


In [9]:
crsr.execute("PRAGMA table_info(shootouts);")
table_info = crsr.fetchall()

for column in table_info:
    print(column)

(0, 'date', 'TEXT', 0, None, 0)
(1, 'home_team', 'TEXT', 0, None, 0)
(2, 'away_team', 'TEXT', 0, None, 0)
(3, 'winner', 'TEXT', 0, None, 0)
(4, 'first_shooter', 'TEXT', 0, None, 0)


# Objectives:

In [10]:
"""
Question 1: 
Create a query that calculates the average number of goals per game 
between 1900 and 2000.
"""

crsr.execute("""
    SELECT AVG(home_score + away_score)
    FROM results
    WHERE date >= '1900-01-01' AND date <= '2000-12-31';
""")
avg_goals = crsr.fetchone()[0]
print(f"Average goals per game between 1900 and 2000: {round(avg_goals, 2)}")

Average goals per game between 1900 and 2000: 3.07


In [11]:
"""
Question 2:
Create a query that counts the number of shootouts wins by country 
and arrange in alphabetical order.
"""

crsr.execute("""
    SELECT winner, COUNT(*) as 'Number of Wins'
    FROM shootouts
    GROUP BY winner
    ORDER BY winner ASC;
""")
print_r(crsr)

+--------------------------+----------------+
|          winner          | Number of Wins |
+--------------------------+----------------+
|         Abkhazia         |       2        |
|         Algeria          |       7        |
|          Angola          |       7        |
|   Antigua and Barbuda    |       2        |
|        Argentina         |       14       |
|          Aruba           |       1        |
|        Australia         |       5        |
|         Bahrain          |       3        |
|        Bangladesh        |       1        |
|         Barbados         |       1        |
|      Basque Country      |       1        |
|         Belarus          |       1        |
|         Belgium          |       2        |
|          Benin           |       3        |
|         Bolivia          |       1        |
|  Bosnia and Herzegovina  |       1        |
|         Botswana         |       8        |
|          Brazil          |       9        |
|  British Virgin Islands  |      

In [12]:
# Identifying duplicates before I add the key
crsr.execute("""
    SELECT date, home_team, away_team, COUNT(*)
    FROM results
    GROUP BY date, home_team, away_team
    HAVING COUNT(*) > 1;
""")
print_r(crsr)

+------------+-----------+---------------+----------+
|    date    | home_team |   away_team   | COUNT(*) |
+------------+-----------+---------------+----------+
| 1974-02-17 |  Tahiti   | New Caledonia |    2     |
| 1977-10-22 |  Guyana   |   Barbados    |    2     |
+------------+-----------+---------------+----------+


In [13]:
crsr.execute("""
    DELETE FROM results 
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM results
        GROUP BY date, home_team, away_team
    );
""")

In [14]:
"""
1. Add a key to results according to date, home-team and away-team
2. Populate to goalscorers and shootouts as a foreign key
"""

# Recreate results with (date, home_team, away_team) key
crsr.execute("""
    DROP TABLE new_results
""")

crsr.execute("""
    CREATE TABLE new_results (
        match_id INTEGER PRIMARY KEY AUTOINCREMENT,
        date TEXT NOT NULL, 
        home_team TEXT NOT NULL, 
        away_team TEXT NOT NULL, 
        home_score INTEGER, 
        away_score INTEGER, 
        tournament TEXT, 
        city TEXT, 
        country TEXT,
        neutral INTEGER
    );
""")

crsr.execute("""
    INSERT INTO new_results (date, home_team, away_team, home_score, away_score, tournament, city, country, neutral)
    SELECT * 
    FROM results;
""")

crsr.execute("""
    DROP TABLE results; 
""")

crsr.execute("""
    ALTER TABLE new_results RENAME to results;
""")


# Recreate goalscorers and shootouts with new foreign key
crsr.execute("""
    CREATE TABLE new_goalscorers (
        match_id INTEGER NOT NULL,
        team TEXT, 
        scorer TEXT,  
        minute REAL, 
        own_goal INTEGER, 
        penalty INTEGER,
        FOREIGN KEY (match_id) REFERENCES results (match_id)
    );
""")

crsr.execute("""
    INSERT INTO new_goalscorers (match_id, team, scorer, minute, own_goal, penalty)
    SELECT
        r.match_id, 
        g.team, 
        g.scorer, 
        g.minute, 
        g.own_goal,
        g.penalty
    FROM goalscorers g
    JOIN results r
    ON g.date = r.date AND g.home_team = r.home_team AND g.away_team = r.away_team
""")

crsr.execute("""
    DROP TABLE goalscorers; 
""")

crsr.execute("""
    ALTER TABLE new_goalscorers RENAME to goalscorers;
""")


crsr.execute("""
    CREATE TABLE new_shootouts (
        match_id INTEGER NOT NULL, 
        winner TEXT, 
        first_shooter TEXT,
        FOREIGN KEY (match_id) REFERENCES results (match_id)
    );
""")

crsr.execute("""
    INSERT INTO new_shootouts (match_id, winner, first_shooter)
    SELECT 
        r.match_id,
        s.winner, 
        s.first_shooter 
    FROM shootouts s
    JOIN results r 
    ON s.date = r.date AND s.home_team = r.home_team AND s.away_team = r.away_team;
""")

crsr.execute("""
    DROP TABLE shootouts; 
""")

crsr.execute("""
    ALTER TABLE new_shootouts RENAME to shootouts;
""")

In [15]:
crsr.execute("""
    SELECT *
    FROM shootouts
    LIMIT 5;
""")

print_r(crsr)

+----------+-------------+---------------+
| match_id |   winner    | first_shooter |
+----------+-------------+---------------+
|   6649   |   Taiwan    |               |
|   8111   | South Korea |               |
|   8282   |    Iraq     |               |
|   8300   | South Korea |               |
|   8303   |  Thailand   |               |
+----------+-------------+---------------+


In [16]:
crsr.execute("""
    SELECT *
    FROM results
    LIMIT 5;
""")

print_r(crsr)

+----------+------------+-----------+-----------+------------+------------+------------+---------+----------+---------+
| match_id |    date    | home_team | away_team | home_score | away_score | tournament |  city   | country  | neutral |
+----------+------------+-----------+-----------+------------+------------+------------+---------+----------+---------+
|    1     | 1872-11-30 | Scotland  |  England  |     0      |     0      |  Friendly  | Glasgow | Scotland |    0    |
|    2     | 1873-03-08 |  England  | Scotland  |     4      |     2      |  Friendly  | London  | England  |    0    |
|    3     | 1874-03-07 | Scotland  |  England  |     2      |     1      |  Friendly  | Glasgow | Scotland |    0    |
|    4     | 1875-03-06 |  England  | Scotland  |     2      |     2      |  Friendly  | London  | England  |    0    |
|    5     | 1876-03-04 | Scotland  |  England  |     3      |     0      |  Friendly  | Glasgow | Scotland |    0    |
+----------+------------+-----------+---

In [17]:
crsr.execute("""
    SELECT *
    FROM goalscorers
    LIMIT 5;
""")

print_r(crsr)

+----------+-----------+------------------+--------+----------+---------+
| match_id |   team    |      scorer      | minute | own_goal | penalty |
+----------+-----------+------------------+--------+----------+---------+
|   439    |  Uruguay  | José Piendibene  |  44.0  |    0     |    0    |
|   439    |  Uruguay  | Isabelino Gradín |  55.0  |    0     |    0    |
|   439    |  Uruguay  | Isabelino Gradín |  70.0  |    0     |    0    |
|   439    |  Uruguay  | José Piendibene  |  75.0  |    0     |    0    |
|   441    | Argentina |  Alberto Ohaco   |  2.0   |    0     |    0    |
+----------+-----------+------------------+--------+----------+---------+


In [18]:
"""
Question 4:
Create a query that identifies which teams have won a penalty 
shootout after a 1-1 draw.
"""
crsr.execute("""
    SELECT winner, COUNT(*) AS "Drawn Game Penalty Wins"
    FROM shootouts s
    JOIN results r
    ON r.match_id = s.match_id
    WHERE r.home_score = 1 AND r.away_score = 1
    GROUP BY winner;
""")

print_r(crsr)

+-------------------------+-------------------------+
|         winner          | Drawn Game Penalty Wins |
+-------------------------+-------------------------+
|        Abkhazia         |            1            |
|         Algeria         |            2            |
|         Angola          |            2            |
|   Antigua and Barbuda   |            1            |
|        Argentina        |            6            |
|         Bahrain         |            1            |
|       Bangladesh        |            1            |
|        Barbados         |            1            |
|     Basque Country      |            1            |
|         Belarus         |            1            |
|         Belgium         |            1            |
|          Benin          |            2            |
|        Botswana         |            2            |
|         Brazil          |            3            |
|        Bulgaria         |            1            |
|      Burkina Faso       | 

In [19]:
"""
Question 5: 
Create a query that identifies the top goal scorer by tournament, and
what percentage that equates to for all goals scored in the tournament.
"""

# Calculate Tournament goals to calculate percentage
# Calculate Scorer goals for maxes

crsr.execute("""
    WITH 
    tournament_goals AS (
        SELECT r.tournament, COUNT(*) AS total_goals
        FROM goalscorers gs
        JOIN results r
        ON r.match_id = gs.match_id
        GROUP BY r.tournament
    ), 
    scorer_goals AS (
        SELECT r.tournament, gs.scorer, COUNT(*) AS scorer_goals
        FROM goalscorers gs
        JOIN results r
        ON r.match_id = gs.match_id
        GROUP BY r.tournament, gs.scorer
)
    SELECT 
        sg.tournament,
        sg.scorer, 
        ROUND((CAST(MAX(sg.scorer_goals) AS FLOAT) / tg.total_goals) * 100.0, 3) AS "Goal Percentage",
        MAX(sg.scorer_goals) as "Number of goals"
    FROM scorer_goals sg
    JOIN tournament_goals tg
    ON sg.tournament = tg.tournament
    GROUP BY sg.tournament
""")
print_r(crsr)

+--------------------------------+-------------------------+-----------------+-----------------+
|           tournament           |         scorer          | Goal Percentage | Number of goals |
+--------------------------------+-------------------------+-----------------+-----------------+
|         AFC Asian Cup          |        Ali Daei         |      1.416      |       14        |
|     African Cup of Nations     |      Samuel Eto'o       |      1.019      |       18        |
|           Baltic Cup           |     Ēriks Pētersons     |      3.93       |        9        |
|   British Home Championship    |       Geoff Hurst       |     12.121      |        4        |
| CONMEBOL–UEFA Cup of Champions |    Claudio Caniggia     |     14.286      |        1        |
|       Confederations Cup       |    Cuauhtémoc Blanco    |      2.128      |        9        |
|          Copa América          | Norberto Doroteo Méndez |      0.636      |       17        |
|         FIFA World Cup      

In [20]:
"""
Additional 
1. Create an additional column that flags records with data quality issues
2. Resolve the identified quality issues
"""

# Empty or duplicate values, if so, set column True

crsr.execute("""
    ALTER TABLE results ADD COLUMN quality_flag INTEGER DEFAULT 0;
""")

crsr.execute("""
    UPDATE results
    SET quality_flag = 1
    WHERE 
        home_score IS NULL OR 
        away_score IS NULL OR 
        tournament IS NULL OR 
        city IS NULL OR 
        country is NULL OR 
        neutral is NULL;
""")

crsr.execute("""
    UPDATE results 
    SET quality_flag = 1
    WHERE rowid NOT IN (
         SELECT MIN(rowid)
         FROM results
         GROUP BY date, home_team, away_team, home_score, away_score, tournament, city, country, neutral
    );
""")


crsr.execute("""
    ALTER TABLE shootouts ADD COLUMN quality_flag INTEGER DEFAULT 0;
""")

# We can have multiple shootouts
crsr.execute("""
    UPDATE shootouts
    SET quality_flag = 1
    WHERE 
        match_id IS NULL OR 
        winner IS NULL OR 
        first_shooter IS NULL; 
""")

crsr.execute("""
    ALTER TABLE goalscorers ADD COLUMN quality_flag INTEGER DEFAULT 0;
""")

crsr.execute("""
    UPDATE goalscorers 
    SET quality_flag = 1
    WHERE 
        team IS NULL OR 
        scorer IS NULL OR 
        minute IS NULL OR 
        own_goal IS NULL OR 
        penalty is NULL; 
""")

crsr.execute("""
    UPDATE goalscorers 
    SET quality_flag = 1
    WHERE rowid NOT IN (
         SELECT MIN(rowid)
         FROM goalscorers
         GROUP BY match_id, team, scorer, minute, own_goal, penalty
    );
""")

In [21]:
crsr.execute("""
    SELECT COUNT(*) as 'Flagged Records in Results'
    FROM results
    WHERE quality_flag == 1;
""")

print_r(crsr)

crsr.execute("""
    SELECT COUNT(*) as 'Flagged Records in Goalscorers'
    FROM goalscorers
    WHERE quality_flag == 1;
""")

print_r(crsr)

crsr.execute("""
    SELECT COUNT(*) as 'Flagged Records in Shootouts'
    FROM shootouts
    WHERE quality_flag == 1;
""")

print_r(crsr)

+----------------------------+
| Flagged Records in Results |
+----------------------------+
|             0              |
+----------------------------+
+--------------------------------+
| Flagged Records in Goalscorers |
+--------------------------------+
|              307               |
+--------------------------------+
+------------------------------+
| Flagged Records in Shootouts |
+------------------------------+
|             475              |
+------------------------------+


In [22]:
crsr.execute("""
    SELECT * 
    FROM goalscorers
    WHERE 
        team IS NULL OR 
        scorer IS NULL OR 
        minute IS NULL OR 
        own_goal IS NULL OR 
        penalty is NULL; 
""")

print_r(crsr)

+----------+---------------------+---------------------------+--------+----------+---------+--------------+
| match_id |        team         |          scorer           | minute | own_goal | penalty | quality_flag |
+----------+---------------------+---------------------------+--------+----------+---------+--------------+
|   4824   |       Taiwan        |       Yiu Cheuk Yin       |        |    0     |    0    |      1       |
|   5557   |        Ghana        |       Edward Acquah       |        |    0     |    0    |      1       |
|   5557   |        Ghana        |       Edward Acquah       |        |    0     |    0    |      1       |
|   5559   |      Ethiopia       |      Mengistu Worku       |        |    0     |    0    |      1       |
|   5559   |      Ethiopia       |      Mengistu Worku       |        |    0     |    0    |      1       |
|   5559   |      Ethiopia       |        Girma Tekle        |        |    0     |    0    |      1       |
|   5559   |      Ethiopia  

In [23]:
crsr.execute("""
    SELECT * 
    FROM shootouts
    WHERE 
        match_id IS NULL OR 
        winner IS NULL OR 
        first_shooter IS NULL; 
""")

print_r(crsr)

+----------+--------------------------+---------------+--------------+
| match_id |          winner          | first_shooter | quality_flag |
+----------+--------------------------+---------------+--------------+
|   6649   |          Taiwan          |               |      1       |
|   8111   |       South Korea        |               |      1       |
|   8282   |           Iraq           |               |      1       |
|   8300   |       South Korea        |               |      1       |
|   8303   |         Thailand         |               |      1       |
|   8695   |          Ghana           |               |      1       |
|   8788   |          Guinea          |               |      1       |
|   8789   |        Mauritius         |               |      1       |
|   8830   |         Malaysia         |               |      1       |
|   8829   |        Singapore         |               |      1       |
|   8832   |         Thailand         |               |      1       |
|   88

In [24]:
crsr.execute("""
    DELETE FROM goalscorers 
    WHERE quality_flag == 1;
""")

crsr.execute("""
    DELETE FROM shootouts
    WHERE quality_flag == 1;
""")

crsr.execute("""
    SELECT COUNT(*) as 'Flagged Records in Goalscorers'
    FROM goalscorers
    WHERE quality_flag == 1;
""")

print_r(crsr)

crsr.execute("""
    SELECT COUNT(*) as 'Flagged Records in Shootouts'
    FROM shootouts
    WHERE quality_flag == 1;
""")

print_r(crsr)

+--------------------------------+
| Flagged Records in Goalscorers |
+--------------------------------+
|               0                |
+--------------------------------+
+------------------------------+
| Flagged Records in Shootouts |
+------------------------------+
|              0               |
+------------------------------+
